# Sinopia testing of `bibframe2marc.xsl`


Sets-up notebook environment and initializes three RDF namespaces and prints the current date.

In [14]:
import datetime
import io
import rdflib
import requests
import pymarc
import sys
from lxml import etree
print(datetime.datetime.utcnow())

BF = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
BFLC = rdflib.Namespace("http://id.loc.gov/ontologies/bflc/")
SINOPIA = rdflib.Namespace("http://sinopia.io/vocabulary/")



2019-12-13 17:48:01.089124


Function binds three of the specific RDF namespaces used by Sinopia.

In [2]:
def init_ns(graph):
    graph.namespace_manager.bind("bf", BF)
    graph.namespace_manager.bind("bflc", BFLC)
    graph.namespace_manager.bind("sinopia", SINOPIA)

In [11]:

# Function takes a list Sinopia URIs and returns a RDF Graph
def triples2record(uris):
    record_graph = rdflib.Graph()
    init_ns(record_graph)
    for uri in uris:
        get_request = requests.get(uri)
        raw_utf8 = get_request.text.encode(get_request.encoding).decode()
        record_graph.parse(data=raw_utf8, format='turtle')
    return record_graph

# Function takes a graph, serializes to XML, and the returns the transformed MARC XML
def graph2marcXML(graph):
    rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
    try:
        return bf2marc_transform(rdf_xml)
    except:
        print(f"Error transforming graph to {sys.exc_info()[0]}")
    
# Function takes a MARC XML document and returns the MARC 21 equalivant
def marcXMLto21(marcXML):
    reader = io.StringIO(etree.tostring(davis_marc_xml).decode())
    marc_records = pymarc.parse_xml_to_array(reader)
    # Should only be 1 record in the list
    return marc_records[0]
    
    

Opens the XSLT as a file and creates an [lxml XSLT](https://lxml.de/xpathxslt.html) object.

In [4]:
with open("../src/xslt/bibframe2marc.xsl", "rb") as fo:
    xml = etree.XML(fo.read())
    bf2marc_transform = etree.XSLT(xml)

# Example -- University of California Davis
Repeat the steps in example one to create a combined BF Work and Instance Graph. Also printing out the number of triples in each graph after it is parsed into the final graph. In the resulting serialization in Turtle you can see the BF Work and Instance is represented as top-level subject edges. The Trellis GET response text was encoded in **ISO-8859-1** and needed to be decoded to **utf-8**.

In [6]:
davis_record = triples2record(["https://trellis.stage.sinopia.io/repository/ucdavis/be99291f-ff37-4dd4-bc71-8ff4ac46d3fb",
                               "https://trellis.stage.sinopia.io/repository/ucdavis/307b37cd-980a-436c-881b-6ed1aa0afeeb"])


print(davis_record.serialize(format='turtle').decode())

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://trellis.stage.sinopia.io/reposi

### RDF-XML Serialization and MARC Transformation
With the new RDF graph called `davis_record`, we can then serialize it as RDF-XML and pass it through `bf2marc_transform` XSLT to produce the MARC XML in `davis_marc_xml`.

In [12]:
davis_marc_xml = graph2marcXML(davis_record)
print(davis_marc_xml)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140491395026712</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">19121310|||||||||xx            ||| |     |</marc:controlfield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">9787502744878</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="245" xml:lang="chi" ind1="1" ind2="0">
    <marc:subfield code="a">中國古動物</marc:subfield>
    <marc:subfield code="c">關键</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="jpn" ind1="1" ind2="1">
    <marc:subfield code="a">中国古動物</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="eng" ind1="1" ind2="1">
    <marc:subfield code="a">The fossil animals of China</marc:subfield>
  </marc:datafield>
  <marc:

Use `pymarc` to create MARC21 format.

In [15]:
marc_record = marcXMLto21(davis_marc_xml)
print(marc_record)

=LDR       nam a22     uu 4500
=001  idp140491395026712
=003  DLC
=008  19121310|||||||||xx\\\\\\\\\\\\|||\|\\\\\|
=020  \\$a9787502744878
=245  10$a中國古動物$c關键
=246  11$a中国古動物
=246  11$aThe fossil animals of China
=250  \\$a第1版
=264  \1$c1998.
=300  \\$a197 pages$c29 cm
=500  \\$aText in Chinese, English, and Japanese ; nomenclature also in Latin
=500  \\$aIncludes index in Latin
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191213104714.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



## Example from Library of Congress
Copied the BIBFRAME RDF-XML from the comparsion tool-kit at (http://id.loc.gov/tools/bibframe/comparebf-lccn/2004701771.xml) and directly parsed and transformed the XML to MARC.

In [22]:

raw_lcc_record = """<rdf:RDF xmlns:bf = "http://id.loc.gov/ontologies/bibframe/" xmlns:bflc = "http://id.loc.gov/ontologies/bflc/" xmlns:madsrdf = "http://www.loc.gov/mads/rdf/v1#" xmlns:rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs = "http://www.w3.org/2000/01/rdf-schema#" >
<bf:Work rdf:about="http://bibframe.example.org/13678831#Work" >
<bf:adminMetadata >
<bf:AdminMetadata >
<bf:generationProcess >
<bf:GenerationProcess >
<rdfs:label >DLC marc2bibframe2 v1.5.1: 2019-12-12T11:11:08-05:00</rdfs:label>
</bf:GenerationProcess>
</bf:generationProcess>
<bf:status >
<bf:Status >
<rdfs:label >corrected or revised</rdfs:label>
<bf:code >c</bf:code>
</bf:Status>
</bf:status>
<bflc:encodingLevel >
<bflc:EncodingLevel rdf:about="http://id.loc.gov/vocabulary/menclvl/4" >
<rdfs:label >core</rdfs:label>
</bflc:EncodingLevel>
</bflc:encodingLevel>
<bf:descriptionConventions >
<bf:DescriptionConventions rdf:about="http://id.loc.gov/vocabulary/descriptionConventions/aacr" >
<rdfs:label >aacr</rdfs:label>
</bf:DescriptionConventions>
</bf:descriptionConventions>
<bf:identifiedBy >
<bf:Local >
<rdf:value >13678831</rdf:value>
<bf:assigner >
<bf:Agent rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:assigner>
</bf:Local>
</bf:identifiedBy>
<bf:changeDate rdf:datatype="http://www.w3.org/2001/XMLSchema#dateTime" >2007-08-25T03:55:05</bf:changeDate>
<bf:creationDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date" >2004-06-18</bf:creationDate>
<bf:source >
<bf:Source >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent" />
<rdfs:label >DLC-R</rdfs:label>
</bf:Source>
</bf:source>
<bf:source >
<bf:Source >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent" />
<rdfs:label >DLC-R</rdfs:label>
</bf:Source>
</bf:source>
<bf:descriptionModifier >
<bf:Agent rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" >
<rdfs:label >DLC</rdfs:label>
</bf:Agent>
</bf:descriptionModifier>
<bf:descriptionAuthentication >
<bf:DescriptionAuthentication rdf:about="http://id.loc.gov/vocabulary/marcauthen/pcc" >
<rdf:value >pcc</rdf:value>
</bf:DescriptionAuthentication>
</bf:descriptionAuthentication>
</bf:AdminMetadata>
</bf:adminMetadata>
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Text" />
<bf:language >
<bf:Language rdf:about="http://id.loc.gov/vocabulary/languages/jpn" />
</bf:language>
<bf:illustrativeContent >
<bf:Illustration rdf:about="http://id.loc.gov/vocabulary/millus/ill" >
<rdfs:label >illustrations</rdfs:label>
</bf:Illustration>
</bf:illustrativeContent>
<bf:supplementaryContent >
<bf:SupplementaryContent rdf:about="http://id.loc.gov/authorities/genreForms/gf2014026048" >
<rdfs:label >bibliographies</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:supplementaryContent >
<bf:SupplementaryContent rdf:about="http://id.loc.gov/vocabulary/msupplcont/index" >
<rdfs:label >Index present</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:identifiedBy >
<bf:Lccn >
<rdf:value >2004701771</rdf:value>
</bf:Lccn>
</bf:identifiedBy>
<bf:geographicCoverage >
<bf:GeographicCoverage rdf:about="http://id.loc.gov/vocabulary/geographicAreas/a-ja" />
</bf:geographicCoverage>
<bf:classification >
<bf:ClassificationLcc >
<bf:source >
<bf:Source rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:source>
<bf:classificationPortion >DS859</bf:classificationPortion>
<bf:itemPortion >.T3626 2004</bf:itemPortion>
</bf:ClassificationLcc>
</bf:classification>
<bf:classification >
<bf:Classification >
<bf:classificationPortion >210.42</bf:classificationPortion>
<bf:source >
<bf:Source >
<rdfs:label >njb/9</rdfs:label>
</bf:Source>
</bf:source>
</bf:Classification>
</bf:classification>
<bf:contribution >
<bf:Contribution >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bflc/PrimaryContribution" />
<bf:agent >
<bf:Agent rdf:about="http://id.loc.gov/authorities/names/nr2004032488" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />
<bflc:name00MatchKey >Takahashi, Kazuki, 1967-</bflc:name00MatchKey>
<bflc:primaryContributorName00MatchKey >Takahashi, Kazuki, 1967-</bflc:primaryContributorName00MatchKey>
<bflc:name00MarcKey >1001 $6880-01$aTakahashi, Kazuki,$d1967-</bflc:name00MarcKey>
<rdfs:label >Takahashi, Kazuki, 1967-</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:role >
<bf:Role rdf:about="http://id.loc.gov/vocabulary/relators/ctb" />
</bf:role>
</bf:Contribution>
</bf:contribution>
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bf:title >
<bf:Title >
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bflc:titleSortKey >Chūsei shōensei to Kamakura Bakufu</bflc:titleSortKey>
<bf:mainTitle >Chūsei shōensei to Kamakura Bakufu</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:supplementaryContent >
<bf:SupplementaryContent >
<rdfs:label >Includes bibliographical references and indexes.</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:subject >
<bf:Place rdf:about="http://id.loc.gov/authorities/subjects/sh85069454" >
<rdf:type rdf:resource="http://www.loc.gov/mads/rdf/v1#ComplexSubject" />
<rdfs:label >Japan--History--Kamakura period, 1185-1333.</rdfs:label>
<madsrdf:authoritativeLabel >Japan--History--Kamakura period, 1185-1333.</madsrdf:authoritativeLabel>
<madsrdf:isMemberOfMADSScheme rdf:resource="http://id.loc.gov/authorities/subjects" />
<madsrdf:componentList rdf:parseType="Collection" >
<madsrdf:Geographic rdf:about="http://id.loc.gov/authorities/names/n78089021" >
<madsrdf:authoritativeLabel >Japan</madsrdf:authoritativeLabel>
</madsrdf:Geographic>
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85061212" >
<madsrdf:authoritativeLabel >History</madsrdf:authoritativeLabel>
</madsrdf:Topic>
<madsrdf:Temporal >
<madsrdf:authoritativeLabel >Kamakura period, 1185-1333</madsrdf:authoritativeLabel>
</madsrdf:Temporal>
</madsrdf:componentList>
<bf:source >
<bf:Source >
<bf:code >lcsh</bf:code>
</bf:Source>
</bf:source>
</bf:Place>
</bf:subject>
<bf:subject >
<bf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh2008107312" >
<rdf:type rdf:resource="http://www.loc.gov/mads/rdf/v1#ComplexSubject" />
<rdfs:label >Manors--Japan--History.</rdfs:label>
<madsrdf:authoritativeLabel >Manors--Japan--History.</madsrdf:authoritativeLabel>
<madsrdf:isMemberOfMADSScheme rdf:resource="http://id.loc.gov/authorities/subjects" />
<madsrdf:componentList rdf:parseType="Collection" >
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85080612" >
<madsrdf:authoritativeLabel >Manors</madsrdf:authoritativeLabel>
</madsrdf:Topic>
<madsrdf:Geographic rdf:about="http://id.loc.gov/authorities/names/n78089021" >
<madsrdf:authoritativeLabel >Japan</madsrdf:authoritativeLabel>
</madsrdf:Geographic>
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85061212" >
<madsrdf:authoritativeLabel >History</madsrdf:authoritativeLabel>
</madsrdf:Topic>
</madsrdf:componentList>
<bf:source >
<bf:Source >
<bf:code >lcsh</bf:code>
</bf:Source>
</bf:source>
</bf:Topic>
</bf:subject>
<bf:contribution >
<bf:Contribution >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bflc/PrimaryContribution" />
<bf:agent >
<bf:Agent rdf:about="http://id.loc.gov/authorities/names/nr2004032488" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />
<bflc:name00MatchKey >高橋一樹, 1967-</bflc:name00MatchKey>
<bflc:primaryContributorName00MatchKey >高橋一樹, 1967-</bflc:primaryContributorName00MatchKey>
<bflc:name00MarcKey >8801 $6100-01/$1$a高橋一樹,$d1967-</bflc:name00MarcKey>
<rdfs:label xml:lang="ja-jpan" >高橋一樹, 1967-</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:role >
<bf:Role rdf:about="http://id.loc.gov/vocabulary/relators/ctb" />
</bf:role>
</bf:Contribution>
</bf:contribution>
<bf:title >
<bf:Title >
<rdfs:label xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</rdfs:label>
<bflc:titleSortKey >中世荘園制と鎌倉幕府</bflc:titleSortKey>
<bf:mainTitle xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:hasInstance rdf:resource="http://bibframe.example.org/13678831#Instance" />
</bf:Work>
<bf:Instance rdf:about="http://bibframe.example.org/13678831#Instance" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print" />
<bf:issuance >
<bf:Issuance rdf:about="http://id.loc.gov/vocabulary/issuance/mono" />
</bf:issuance>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:date rdf:datatype="http://id.loc.gov/datatypes/edtf" >2004</bf:date>
<bf:place >
<bf:Place rdf:about="http://id.loc.gov/vocabulary/countries/ja" />
</bf:place>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:identifiedBy >
<bf:Lccn >
<rdf:value >2004701771</rdf:value>
</bf:Lccn>
</bf:identifiedBy>
<bf:identifiedBy >
<bf:Isbn >
<rdf:value >4827311838</rdf:value>
</bf:Isbn>
</bf:identifiedBy>
<bf:identifiedBy >
<bf:Local >
<rdf:value >DCLP04-B11050</rdf:value>
<bf:source >
<bf:Source >
<rdfs:label >CStRLIN</rdfs:label>
</bf:Source>
</bf:source>
</bf:Local>
</bf:identifiedBy>
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bf:title >
<bf:Title >
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bflc:titleSortKey >Chūsei shōensei to Kamakura Bakufu</bflc:titleSortKey>
<bf:mainTitle >Chūsei shōensei to Kamakura Bakufu</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:responsibilityStatement >Takahashi Kazuki cho</bf:responsibilityStatement>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:place >
<bf:Place >
<rdfs:label >Tōkyō</rdfs:label>
</bf:Place>
</bf:place>
<bf:agent >
<bf:Agent >
<rdfs:label >Hanawa Shobō</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:date >2004</bf:date>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:provisionActivityStatement >Tōkyō : Hanawa Shobō, 2004.</bf:provisionActivityStatement>
<bf:extent >
<bf:Extent >
<rdfs:label >vii, 433, 31 p.</rdfs:label>
</bf:Extent>
</bf:extent>
<bf:note >
<bf:Note >
<bf:noteType >Physical details</bf:noteType>
<rdfs:label >ill.</rdfs:label>
</bf:Note>
</bf:note>
<bf:dimensions >22 cm.</bf:dimensions>
<bf:title >
<bf:Title >
<rdfs:label xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</rdfs:label>
<bflc:titleSortKey >中世荘園制と鎌倉幕府</bflc:titleSortKey>
<bf:mainTitle xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:responsibilityStatement xml:lang="ja-jpan" >高橋一樹著</bf:responsibilityStatement>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:place >
<bf:Place >
<rdfs:label xml:lang="ja-jpan" >東京</rdfs:label>
</bf:Place>
</bf:place>
<bf:agent >
<bf:Agent >
<rdfs:label xml:lang="ja-jpan" >塙書房</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:date xml:lang="ja-jpan" >2004</bf:date>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:provisionActivityStatement xml:lang="ja-jpan" >東京 : 塙書房, 2004.</bf:provisionActivityStatement>
<bf:instanceOf rdf:resource="http://bibframe.example.org/13678831#Work" />
<bf:hasItem >
<bf:Item rdf:about="http://bibframe.example.org/13678831#Item050-14" >
<bf:shelfMark >
<bf:ShelfMarkLcc >
<rdfs:label >DS859.T3626 2004</rdfs:label>
<bf:source >
<bf:Source rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:source>
</bf:ShelfMarkLcc>
</bf:shelfMark>
<bf:itemOf rdf:resource="http://bibframe.example.org/13678831#Instance" />
</bf:Item>
</bf:hasItem>
</bf:Instance>
</rdf:RDF>"""

In [23]:
loc_marc = bf2marc_transform(etree.XML(raw_lcc_record))

In [24]:
print(loc_marc)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     cam a22     4a 4500</marc:leader>
  <marc:controlfield tag="001">13678831</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="005">20070825035505.0</marc:controlfield>
  <marc:controlfield tag="008">040618s2004    ja            ||| | jpn |</marc:controlfield>
  <marc:datafield tag="010" ind1=" " ind2=" ">
    <marc:subfield code="a">2004701771</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">4827311838</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="035" ind1=" " ind2=" ">
    <marc:subfield code="a">(CStRLIN)DCLP04-B11050</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="040" ind1=" " ind2=" ">
    <marc:subfield code="a">DLC-R</marc:subfield>
    <marc:subfield code="c">DLC-R</marc:subfield>
    <marc:subfiel